In [5]:
import numpy as np
import cv2
face_classifier = cv2.CascadeClassifier('C:/Users/Lenovo/Desktop/mlops-ws/haarcascade_frontalface_default.xml')
def face_extractor(img):
    faces = face_classifier.detectMultiScale(img,1.3,5)
    if faces is ():
        return None
    for(x,y,w,h) in faces:
          cropped_face = img[y:y+h , x:x+w]
    return  cropped_face
cap = cv2.VideoCapture(0)
count = 0
while True:
        ret,frame = cap.read()
        if face_extractor(frame) is not None:
            count +=1
            face = cv2.resize(face_extractor(frame), (224,224))
            file_name_path = 'C://Users//Lenovo//Desktop//imagedata//train//n0//'  + str(count) + '.jpg'
            cv2.imwrite(file_name_path , face)
            cv2.putText(face, str(count) , (100,100) , cv2.FONT_HERSHEY_COMPLEX , 1, (0,255,0) , 2)
            cv2.imshow('face cropper',face)
        else:
            print("Face not found")
            pass
        if cv2.waitKey(1) == 13 or count == 200:
            break
cap.release()
cv2.destroyAllWindows()
print("Sample Collection Completed")


Sample Collection Completed


In [20]:
from keras.applications import VGG16

In [21]:
from keras.applications import VGG16

In [22]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [23]:
def addTopModel(bottom_model, num_classes , D = 256):
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D , activation = 'relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation ="softmax")(top_model)
    return top_model

In [24]:
model.input

<tf.Tensor 'input_2:0' shape=(?, 224, 224, 3) dtype=float32>

In [25]:
model.layers

In [26]:
from tensorflow.keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs = model.input, outputs = FC_Head)

print(modelnew.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [27]:
from keras.preprocessing.image import ImageDataGenerator

In [28]:
train_data_dir = 'C:/Users/Lenovo/Desktop/imagedata/train/'
validation_data_dir = 'C:/Users/Lenovo/Desktop/imagedata/test/'
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 20,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = True,
        fill_mode = 'nearest')

In [29]:
validation_datagen = ImageDataGenerator(rescale = 1./225)
train_batchsize = 16
val_batchsize = 10
train_generator = train_datagen.flow_from_directory(
               train_data_dir,
               target_size = (img_rows,img_cols),
               batch_size = train_batchsize,
               class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory(
              validation_data_dir,
              target_size = (img_rows, img_cols),
              batch_size = val_batchsize,
              class_mode = 'categorical',
              shuffle = False)

Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [30]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face_recognition.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 3
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("face_recognition.h5")


Epoch 1/3
74/74 [==============================] - 1073s 14s/step - loss: 3246.3448 - accuracy: 0.4688 - val_loss: 0.6375 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.63747, saving model to face_recognition.h5
Epoch 2/3
74/74 [==============================] - 1072s 14s/step - loss: 0.6938 - accuracy: 0.4797 - val_loss: 0.6957 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.63747
Epoch 3/3
74/74 [==============================] - 1115s 15s/step - loss: 0.6934 - accuracy: 0.4772 - val_loss: 0.6900 - val_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.63747


In [ ]:
from keras.models import load_model

classifier = load_model('face_recognition.h5')


# In[14]:


import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

face_recognition = {"[0]": "Pranali Joshi", 
                    "[1]": "Pranali's Mother"}
face_recognition_n = {"n0": "Pranali Joshi", 
                      "n1": "Pranali's Mother"}
def draw_test(name, pred, im):
    face = face_recognition[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_recognition_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("C:/Users/Lenovo/Desktop/imagedata/train/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1) 
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)
cv2.destroyAllWindows()